# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#500 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846833251148                   -0.70    4.0
  2   -7.852321498452       -2.26       -1.53    1.0
  3   -7.852646032278       -3.49       -2.52    3.2
  4   -7.852646677374       -6.19       -3.33    2.2
  5   -7.852646686042       -8.06       -4.78    1.2
  6   -7.852646686728       -9.16       -5.31    4.2
  7   -7.852646686730      -11.71       -5.97    1.0
  8   -7.852646686730      -12.62       -7.47    2.0
  9   -7.852646686730   +  -14.75       -7.68    3.0
 10   -7.852646686730      -15.05       -8.24    1.2
 11   -7.852646686730      -15.05       -9.27    2.0
 12   -7.852646686730   +    -Inf       -9.99    2.2
 13   -7.852646686730   +  -14.75      -10.83    2.8
 14   -7.852646686730   +    -Inf      -11.82    2.8
 15   -7.852646686730   +  -14.75      -12.80    2.2


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846833449932                   -0.70           4.2
  2   -7.852527195495       -2.24       -1.62   0.80    4.8
  3   -7.852608773304       -4.09       -2.78   0.80    5.5
  4   -7.852646575605       -4.42       -3.57   0.80    7.5
  5   -7.852646683692       -6.97       -4.62   0.80    8.8
  6   -7.852646686689       -8.52       -5.17   0.80   10.8
  7   -7.852646686729      -10.40       -6.25   0.80   11.8
  8   -7.852646686730      -11.92       -7.47   0.80   13.2
  9   -7.852646686730      -13.65       -7.68   0.80   15.8
 10   -7.852646686730   +  -14.57       -9.34   0.80   15.8
 11   -7.852646686730      -14.75       -9.27   0.80   18.8
 12   -7.852646686730   +  -14.75      -10.59   0.80   19.0
 13   -7.852646686730      -14.75      -11.00   0.80   21.2
 14   -7.852646686730      -14.75      -12.10   0.80   22.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.376487e+01     3.824975e+00
 * time: 0.44709300994873047
     1     1.046923e+00     1.726901e+00
 * time: 0.666165828704834
     2    -1.046241e+00     1.968882e+00
 * time: 0.6876709461212158
     3    -3.504372e+00     1.802542e+00
 * time: 0.7181839942932129
     4    -4.675531e+00     1.548446e+00
 * time: 0.7487139701843262
     5    -6.485122e+00     9.468604e-01
 * time: 0.7793979644775391
     6    -6.496113e+00     1.265374e+00
 * time: 0.8008549213409424
     7    -7.284944e+00     8.397349e-01
 * time: 0.8223159313201904
     8    -7.645253e+00     2.459877e-01
 * time: 0.8534529209136963
     9    -7.748649e+00     1.382480e-01
 * time: 0.8750309944152832
    10    -7.797257e+00     1.107824e-01
 * time: 0.8965630531311035
    11    -7.827773e+00     8.544674e-02
 * time: 0.9179668426513672
    12    -7.841810e+00     4.907677e-02
 * time: 0.9399139881134033
    13    -7.848662e+00     2.421748e-02
 * time: 0.9618768692

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846826141961                   -0.70    4.0
  2   -7.852305103922       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686239                   -2.55
  2   -7.852646686730       -9.31       -5.43
  3   -7.852646686730   +    -Inf      -10.44
  4   -7.852646686730   +  -14.45      -15.44
|ρ_newton - ρ_scf|  = 1.830610593029754e-13
|ρ_newton - ρ_scfv| = 7.590082556982906e-13
|ρ_newton - ρ_dm|   = 6.951170667248918e-10
